In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import svm, neighbors, ensemble
from sklearn.metrics import classification_report, accuracy_score
import joblib
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import GridSearchCV

C:\Users\Sumking007\AppData\Local\Temp\ipykernel_7072\1611314746.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_dir = "F:\my pro\deep_and_fake"

real_dir = os.path.join(data_dir, "real_images")
fake_dir = os.path.join(data_dir, "deepfake_images")

real_images = os.listdir(real_dir)
fake_images = os.listdir(fake_dir)

data = []
labels = []

for img in real_images:
    img_path = os.path.join(real_dir, img)
    img_data = cv2.imread(img_path)
    img_data = cv2.resize(img_data, (100, 100))
    data.append(img_data)
    labels.append("Real")

for img in fake_images:
    img_path = os.path.join(fake_dir, img)
    img_data = cv2.imread(img_path)
    img_data = cv2.resize(img_data, (100, 100))
    data.append(img_data)
    labels.append("Deepfake")

data = np.array(data)
labels = np.array(labels)

# Normalize the image data
data = data / 255.0

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

In [14]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=4, batch_size=32, validation_data=(X_test, y_test))

# Save the trained model for feature extraction
model.save("cnn_model.h5")

Epoch 1/4
273/273 [==============================] - 167s 503ms/step - loss: 0.5899 - accuracy: 0.6753 - val_loss: 0.5658 - val_accuracy: 0.7056
Epoch 2/4
273/273 [==============================] - 94s 345ms/step - loss: 0.5050 - accuracy: 0.7397 - val_loss: 0.4886 - val_accuracy: 0.7561
Epoch 3/4
273/273 [==============================] - 94s 343ms/step - loss: 0.4411 - accuracy: 0.7839 - val_loss: 0.4489 - val_accuracy: 0.7895
Epoch 4/4
273/273 [==============================] - 93s 342ms/step - loss: 0.3772 - accuracy: 0.8276 - val_loss: 0.3739 - val_accuracy: 0.8322


c:\Users\Sumking007\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
cnn_model = tf.keras.models.load_model("cnn_model.h5")

train_features = cnn_model.predict(X_train)
test_features = cnn_model.predict(X_test)

69/69 [==============================] - 8s 108ms/step


In [16]:
# SVM
svm_model = svm.SVC()
svm_model.fit(train_features, y_train)

# KNN
knn_model = neighbors.KNeighborsClassifier()
knn_model.fit(train_features, y_train)

# Random Forest
rf_model = ensemble.RandomForestClassifier()
rf_model.fit(train_features, y_train)

RandomForestClassifier()

In [17]:
# SVM
svm_params = {'C': [1, 10, 100], 'kernel': ['linear', 'rbf']}
svm_tuned = GridSearchCV(estimator=svm_model, param_grid=svm_params, scoring='accuracy', cv=3)
svm_tuned.fit(train_features, y_train)
svm_best = svm_tuned.best_estimator_

svm_pred = svm_best.predict(test_features)
svm_accuracy = accuracy_score(y_test, svm_pred)
svm_classification_report = classification_report(y_test, svm_pred)

# KNN
knn_params = {'n_neighbors': [3, 5, 7]}
knn_tuned = GridSearchCV(estimator=knn_model, param_grid=knn_params, scoring='accuracy', cv=3)
knn_tuned.fit(train_features, y_train)
knn_best = knn_tuned.best_estimator_

knn_pred = knn_best.predict(test_features)
knn_accuracy = accuracy_score(y_test, knn_pred)
knn_classification_report = classification_report(y_test, knn_pred)

# Random Forest
rf_params = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]}
rf_tuned = GridSearchCV(estimator=rf_model, param_grid=rf_params, scoring='accuracy', cv=3)
rf_tuned.fit(train_features, y_train)
rf_best = rf_tuned.best_estimator_

rf_pred = rf_best.predict(test_features)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_classification_report = classification_report(y_test, rf_pred)

print("SVM Accuracy:", svm_accuracy)
print("SVM Classification Report:\n", svm_classification_report)

print("KNN Accuracy:", knn_accuracy)
print("KNN Classification Report:\n", knn_classification_report)

print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Classification Report:\n", rf_classification_report)

SVM Accuracy: 0.8303530490600642
SVM Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.80      0.82      1093
           1       0.81      0.87      0.84      1088

    accuracy                           0.83      2181
   macro avg       0.83      0.83      0.83      2181
weighted avg       0.83      0.83      0.83      2181

KNN Accuracy: 0.8165978908757451
KNN Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.79      0.81      1093
           1       0.80      0.84      0.82      1088

    accuracy                           0.82      2181
   macro avg       0.82      0.82      0.82      2181
weighted avg       0.82      0.82      0.82      2181

Random Forest Accuracy: 0.823016964695094
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.79      0.82      1093
           1       0.80      0.85  

In [18]:
# Choose the best model based on the highest accuracy
best_model = svm_best if svm_accuracy > knn_accuracy and svm_accuracy > rf_accuracy else knn_best if knn_accuracy > rf_accuracy else rf_best

# Save the best model
joblib.dump(best_model, "best_model.joblib")

['best_model.joblib']

In [22]:
from flask import Flask, render_template, request

app = Flask(__name__)

model = joblib.load("best_model.joblib")

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        file = request.files["file"]
        if file:
            image = cv2.imdecode(np.frombuffer(file.read(), np.uint8), 1)
            image = cv2.resize(image, (100, 100))
            image = image / 255.0
            image_features = cnn_model.predict(np.array([image]))
            prediction = le.inverse_transform(model.predict(image_features))[0]
            return render_template("index.html", prediction=prediction)
        else:
            return render_template("index.html")
    else:
        return render_template("index.html")


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000


Press CTRL+C to quit
127.0.0.1 - - [15/Feb/2024 21:23:31] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 113ms/step


127.0.0.1 - - [15/Feb/2024 21:24:51] "POST / HTTP/1.1" 200 -


...

127.0.0.1 - - [15/Feb/2024 21:27:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2024 21:27:57] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 1s 1s/step


127.0.0.1 - - [15/Feb/2024 21:28:07] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2024 21:29:18] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 225ms/step


127.0.0.1 - - [15/Feb/2024 21:29:25] "POST / HTTP/1.1" 200 -
